In [101]:
from pyspark.sql import SparkSession
session = SparkSession.builder.appName('bigdata').getOrCreate()
df = session.read.csv('dataset/bigdata.csv',header=True,inferSchema= True)
df.show(n=10)

+--------+-----+----+-----+
|    date|store|item|sales|
+--------+-----+----+-----+
|01/01/13|    1|   1|   13|
|02/01/13|    1|   1|   11|
|03/01/13|    1|   1|   14|
|04/01/13|    1|   1|   13|
|05/01/13|    1|   1|   10|
|06/01/13|    1|   1|   12|
|07/01/13|    1|   1|   10|
|08/01/13|    1|   1|    9|
|09/01/13|    1|   1|   12|
|10/01/13|    1|   1|    9|
+--------+-----+----+-----+
only showing top 10 rows



In [102]:
## count row
df.count()

913000

In [103]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- store: integer (nullable = true)
 |-- item: integer (nullable = true)
 |-- sales: integer (nullable = true)



In [104]:
df = df.na.drop(how='any')
df.count()
col_name = df.columns
col_name

['date', 'store', 'item', 'sales']

## we need to modify date field by Replace Column Value Character by Character (/ by -)

By using translate() string function you can replace character by character of DataFrame column value.

In [105]:
###very important code
from pyspark.sql.functions import translate
df = df.withColumn('date', translate('date', '/', '-'))
df.show(n=10)
#This unix_timestamp support only dd-mm-yy ex (02-12-21). but our row data was some of them this format somr of them
#10/12/2021 format. so at frist I fortamt this to 10/12/2021 format in excel. then I replaced / by - using this line. then 
# I apply from_unixtime funsion and change to 

+--------+-----+----+-----+
|    date|store|item|sales|
+--------+-----+----+-----+
|01-01-13|    1|   1|   13|
|02-01-13|    1|   1|   11|
|03-01-13|    1|   1|   14|
|04-01-13|    1|   1|   13|
|05-01-13|    1|   1|   10|
|06-01-13|    1|   1|   12|
|07-01-13|    1|   1|   10|
|08-01-13|    1|   1|    9|
|09-01-13|    1|   1|   12|
|10-01-13|    1|   1|    9|
+--------+-----+----+-----+
only showing top 10 rows



In [106]:
#seperate datetime
from pyspark.sql.functions import year
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.functions import col
from pyspark.sql.functions import unix_timestamp, from_unixtime

df2 = df.select(
    'date', 'store', 'item', 'sales', 
    from_unixtime(unix_timestamp('date', 'dd-mm-yy')).alias('datetime')
)
df2.show(n=10)


+--------+-----+----+-----+-------------------+
|    date|store|item|sales|           datetime|
+--------+-----+----+-----+-------------------+
|01-01-13|    1|   1|   13|2013-01-01 00:01:00|
|02-01-13|    1|   1|   11|2013-01-02 00:01:00|
|03-01-13|    1|   1|   14|2013-01-03 00:01:00|
|04-01-13|    1|   1|   13|2013-01-04 00:01:00|
|05-01-13|    1|   1|   10|2013-01-05 00:01:00|
|06-01-13|    1|   1|   12|2013-01-06 00:01:00|
|07-01-13|    1|   1|   10|2013-01-07 00:01:00|
|08-01-13|    1|   1|    9|2013-01-08 00:01:00|
|09-01-13|    1|   1|   12|2013-01-09 00:01:00|
|10-01-13|    1|   1|    9|2013-01-10 00:01:00|
+--------+-----+----+-----+-------------------+
only showing top 10 rows



In [108]:
df2.printSchema()

root
 |-- date: string (nullable = true)
 |-- store: integer (nullable = true)
 |-- item: integer (nullable = true)
 |-- sales: integer (nullable = true)
 |-- datetime: string (nullable = true)



## Separate datetime : day ,month, year 

In [113]:
df2 = df2.withColumn('month',month(df2.datetime)).withColumn('year',year(df2.datetime))
df2.show()

+--------+-----+----+-----+-------------------+-----+----+
|    date|store|item|sales|           datetime|month|year|
+--------+-----+----+-----+-------------------+-----+----+
|01-01-13|    1|   1|   13|2013-01-01 00:01:00|    1|2013|
|02-01-13|    1|   1|   11|2013-01-02 00:01:00|    1|2013|
|03-01-13|    1|   1|   14|2013-01-03 00:01:00|    1|2013|
|04-01-13|    1|   1|   13|2013-01-04 00:01:00|    1|2013|
|05-01-13|    1|   1|   10|2013-01-05 00:01:00|    1|2013|
|06-01-13|    1|   1|   12|2013-01-06 00:01:00|    1|2013|
|07-01-13|    1|   1|   10|2013-01-07 00:01:00|    1|2013|
|08-01-13|    1|   1|    9|2013-01-08 00:01:00|    1|2013|
|09-01-13|    1|   1|   12|2013-01-09 00:01:00|    1|2013|
|10-01-13|    1|   1|    9|2013-01-10 00:01:00|    1|2013|
|11-01-13|    1|   1|    9|2013-01-11 00:01:00|    1|2013|
|12-01-13|    1|   1|    7|2013-01-12 00:01:00|    1|2013|
|13-01-13|    1|   1|   10|2013-01-13 00:01:00|    1|2013|
|14-01-13|    1|   1|   12|2013-01-14 00:01:00|    1|201

In [115]:
df2.columns

['date', 'store', 'item', 'sales', 'datetime', 'month', 'year']

In [117]:
## Easy way to retrive day

df2 = df2.select(
    'date', 'store', 'item', 'sales', 'datetime', 'month', 'year', 
    dayofmonth("datetime").alias('day')
)
df2.show(n=10)

+--------+-----+----+-----+-------------------+-----+----+---+
|    date|store|item|sales|           datetime|month|year|day|
+--------+-----+----+-----+-------------------+-----+----+---+
|01-01-13|    1|   1|   13|2013-01-01 00:01:00|    1|2013|  1|
|02-01-13|    1|   1|   11|2013-01-02 00:01:00|    1|2013|  2|
|03-01-13|    1|   1|   14|2013-01-03 00:01:00|    1|2013|  3|
|04-01-13|    1|   1|   13|2013-01-04 00:01:00|    1|2013|  4|
|05-01-13|    1|   1|   10|2013-01-05 00:01:00|    1|2013|  5|
|06-01-13|    1|   1|   12|2013-01-06 00:01:00|    1|2013|  6|
|07-01-13|    1|   1|   10|2013-01-07 00:01:00|    1|2013|  7|
|08-01-13|    1|   1|    9|2013-01-08 00:01:00|    1|2013|  8|
|09-01-13|    1|   1|   12|2013-01-09 00:01:00|    1|2013|  9|
|10-01-13|    1|   1|    9|2013-01-10 00:01:00|    1|2013| 10|
+--------+-----+----+-----+-------------------+-----+----+---+
only showing top 10 rows



 ## Separate datetime : day ,month, year (Easy Way)

In [ ]:
# initailize SparkSession and load data

In [120]:
df = df.na.drop(how='any')
df.count()
col_name = df.columns
col_name

['date', 'store', 'item', 'sales']

In [121]:
###very important code
from pyspark.sql.functions import translate
df = df.withColumn('date', translate('date', '/', '-'))
df.show(n=10)
#This unix_timestamp support only dd-mm-yy ex (02-12-21). but our row data was some of them this format somr of them
#10/12/2021 format. so at frist I fortamt this to 10/12/2021 format in excel. then I replaced / by - using this line. then 
# I apply from_unixtime funsion and change to 

+--------+-----+----+-----+
|    date|store|item|sales|
+--------+-----+----+-----+
|01-01-13|    1|   1|   13|
|02-01-13|    1|   1|   11|
|03-01-13|    1|   1|   14|
|04-01-13|    1|   1|   13|
|05-01-13|    1|   1|   10|
|06-01-13|    1|   1|   12|
|07-01-13|    1|   1|   10|
|08-01-13|    1|   1|    9|
|09-01-13|    1|   1|   12|
|10-01-13|    1|   1|    9|
+--------+-----+----+-----+
only showing top 10 rows



In [122]:
#seperate datetime
from pyspark.sql.functions import year
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.functions import col
from pyspark.sql.functions import unix_timestamp, from_unixtime

df2 = df.select(
    'date', 'store', 'item', 'sales', 
    from_unixtime(unix_timestamp('date', 'dd-mm-yy')).alias('datetime')
)
df2.show(n=10)


+--------+-----+----+-----+-------------------+
|    date|store|item|sales|           datetime|
+--------+-----+----+-----+-------------------+
|01-01-13|    1|   1|   13|2013-01-01 00:01:00|
|02-01-13|    1|   1|   11|2013-01-02 00:01:00|
|03-01-13|    1|   1|   14|2013-01-03 00:01:00|
|04-01-13|    1|   1|   13|2013-01-04 00:01:00|
|05-01-13|    1|   1|   10|2013-01-05 00:01:00|
|06-01-13|    1|   1|   12|2013-01-06 00:01:00|
|07-01-13|    1|   1|   10|2013-01-07 00:01:00|
|08-01-13|    1|   1|    9|2013-01-08 00:01:00|
|09-01-13|    1|   1|   12|2013-01-09 00:01:00|
|10-01-13|    1|   1|    9|2013-01-10 00:01:00|
+--------+-----+----+-----+-------------------+
only showing top 10 rows



In [128]:
final_df = df2.select(
    'date','datetime',
    year("datetime").alias('year'), 
    month("datetime").alias('month'),
    dayofmonth("datetime").alias('day'),
    'store', 'item', 'sales',
)
final_df.show(n=10)

+--------+-------------------+----+-----+---+-----+----+-----+
|    date|           datetime|year|month|day|store|item|sales|
+--------+-------------------+----+-----+---+-----+----+-----+
|01-01-13|2013-01-01 00:01:00|2013|    1|  1|    1|   1|   13|
|02-01-13|2013-01-02 00:01:00|2013|    1|  2|    1|   1|   11|
|03-01-13|2013-01-03 00:01:00|2013|    1|  3|    1|   1|   14|
|04-01-13|2013-01-04 00:01:00|2013|    1|  4|    1|   1|   13|
|05-01-13|2013-01-05 00:01:00|2013|    1|  5|    1|   1|   10|
|06-01-13|2013-01-06 00:01:00|2013|    1|  6|    1|   1|   12|
|07-01-13|2013-01-07 00:01:00|2013|    1|  7|    1|   1|   10|
|08-01-13|2013-01-08 00:01:00|2013|    1|  8|    1|   1|    9|
|09-01-13|2013-01-09 00:01:00|2013|    1|  9|    1|   1|   12|
|10-01-13|2013-01-10 00:01:00|2013|    1| 10|    1|   1|    9|
+--------+-------------------+----+-----+---+-----+----+-----+
only showing top 10 rows



In [129]:
final_df.count()

913000

In [132]:
### export data to csv
final_df.toPandas().to_csv('dataset/modyfiedBigdata.csv')
